# DS400: Assignment 5 (Python)

##### Tingyu Lin (Student ID: 5638679)


In [1]:
import requests
import numpy as np
import pandas as pd
import time

In [2]:
# hyperlink: https://github.com/ltyita/Python_Assign05.git

In [8]:
# Ex6 source API key from another script
with open("ticket_APIkey.py") as script:
    exec(script.readline())

### Exercise 7

In [11]:
from IPython.display import Markdown as md

# Exercise (7)
# first GET request that searches for event venues in Germany
response = requests.get("https://app.ticketmaster.com/discovery/v2/venues.json?",
                        params = {"countryCode": "DE", "apikey" : api_key})

# extract content
DE_content = response.json()


# number of page and observation
n = DE_content['page']
n


{'size': 20, 'totalElements': 4750, 'totalPages': 238, 'number': 0}

In [13]:
md(f'From data frame, it suggests that each page contains 20 observations,\
and there are {n["totalElements"]} events in Germany, which make them {n["totalPages"]}-page long. And we are currently at page 0.')


From data frame, it suggests that each page contains 20 observations,and there are 4750 events in Germany, which make them 238-page long. And we are currently at page 0.

### Exercise 8

In [15]:
# Exercise (8)
# make the first 20 obs as a data frame that contains name, the city, the postalCode and address, 
# as well as the url and the longitude and latitude. 

# create an empty data frame dim: 20X7
de_venues = pd.DataFrame(index = range(20), columns = range(7))

# assign col. names
de_venues.columns = ["name", "city", "postalCode", "address", "url",
                     "longitude", "latitude"]


for i in range(0, 20):
    try:
        de_name = DE_content["_embedded"]["venues"][i]["name"]
    except KeyError:
        de_name = np.nan
        
    try:
        de_city = DE_content["_embedded"]["venues"][i]["city"]["name"]
    except KeyError:
        de_city = np.nan
    
    try:
        de_postalCode = DE_content["_embedded"]["venues"][i]["postalCode"]
    except KeyError:
        de_postalCode = np.nan
    
    try:
        de_address = DE_content["_embedded"]["venues"][i]["address"]["line1"]
    except KeyError:
        de_address = np.nan
    
    try:
        de_url = DE_content["_embedded"]["venues"][i]["url"]
    except KeyError:
        de_url = np.nan
    
    try:
        de_longitude = DE_content["_embedded"]["venues"][i]["location"]["longitude"]
    except KeyError:
        de_longitude = np.nan
    
    try:
        de_latitude = DE_content["_embedded"]["venues"][i]["location"]["latitude"]
    except KeyError:
        de_latitude = np.nan
    
    de_venues.iloc[i,:] = [de_name, de_city, de_postalCode, de_address,
                           de_url, de_longitude, de_latitude]   


de_venues

,name,city,postalCode,address,url,longitude,latitude
0,Grosse Freiheit 36,Hamburg,22767,Grosse Freiheit 36,http://www.ticketmaster.de/venue/287636,9.957905,53.551187
1,Gruenspan,Hamburg,22767,Grosse Freiheit 58,http://www.ticketmaster.de/venue/287155,9.958075,53.551885
2,Huxleys Neue Welt,Berlin,10967,Hasenheide 107 – 113,http://www.ticketmaster.de/venue/286842,13.42138,52.486391
3,Virtual Event,Worldwide,NaN,NaN,https://www.ticketmaster.de/venue/virtuelles-e...,10.0,50.0
4,Mojo Club,Hamburg,20359,Reeperbahn 1,https://www.ticketmaster.de/venue/mojo-club-ha...,9.64719,53.75287
5,Ev. St. Jacobi Kirche,Sangerhausen,06526,Marktplatz,http://www.ticketmaster.de/venue/290061,NaN,NaN
6,Evangelische Kirche,Senden,48308,Steverstrasse 5,http://www.ticketmaster.de/venue/290066,NaN,NaN
7,HDI Arena,Hannover,NaN,Robert-Enke-Straße 1,http://www.ticketmaster.de/venue/461692,9.73371,52.361993
8,Arsenal,Berlin,10785,Potsdamer Strasse 2,http://www.ticketmaster.de/venue/290646,NaN,NaN
9,Freilichtbühne Heppenheim,Heppenheim,64646,Oberhalb der Stadt,http://www.ticketmaster.de/venue/290639,NaN,NaN
